In [1]:
import pandas as pd
import os
import glob
import matplotlib.pyplot as plt
import seaborn as sns

In [64]:

def read_table(path):
    df = pd.DataFrame(columns=['model', 'lDDToligo','TMscore','QSglob'])
    with open(path, "r") as f:
        lines = f.read().splitlines()
    for line in lines:
        cols = line.split()
        
        # if line[0] is a string digit
        if not (line[0]).isdigit():
                    
            continue
        if len(cols) < 10:
            print("skipping ", line[0])    
            continue
        model_name = cols[1]
        try:
            lDDToligo = float(cols[16])
            i = -1
            while True:
                if ":" in cols[i]:
                    i -= 1
                else:
                    break
            tmscore = float(cols[i])
            QSglob = float(cols[4])
            
        except ValueError:
            continue
        
        df = df.append({'model': model_name, 'lDDToligo': lDDToligo, 'TMscore': tmscore, 'QSglob': QSglob}, ignore_index=True)
    return df

# read_table("/root/EnQA/casp15_true_results/T1132o.txt")

- Read the true and predict score tables and combine them into one table.
- The output table are saved in 'casp15_qa_comparison'

In [65]:
true_dir = "casp15_true_results"
pred_dir = "casp15_qa_results"
save_dir = 'casp15_qa_comparison'
no_qa_results = []
for table in os.listdir(true_dir):
    if not table.endswith(".txt"):
        continue
    target_name = table.split(".")[0]
    true_df = read_table(os.path.join(true_dir, table))
    pred_table_path = os.path.join(pred_dir, target_name,f"qa.csv")
    if not os.path.exists(pred_table_path):
        no_qa_results.append(pred_table_path)
        continue
    pred_table = pd.read_csv(pred_table_path)
    if pred_table.shape[0] < 100:
        # remove targets that have < 100 predicted models
        continue
    # rename column names
    pred_table.rename(columns={"model_name": "model","lddt":"lDDToligo"}, inplace=True)
    # remove a string "_merged" from the column "model"
    pred_table["model"] = pred_table["model"].str.replace("_merged","")

    shared_df = true_df.merge(pred_table, on='model', how='inner', suffixes=('_True', '_EnQA-MSA'))
    # plot scatter plot for column "lDDToligo_True" and "lDDToligo_EnQA-MSA" and regression line using sns
    sns.scatterplot(x="lDDToligo_True", y="lDDToligo_EnQA-MSA",  data=shared_df)
    sns.regplot(x="lDDToligo_True", y="lDDToligo_EnQA-MSA", data=shared_df)
    plt.savefig(os.path.join(save_dir, f"{target_name}.png"))
    plt.clf()
    
    shared_df.to_csv(os.path.join(save_dir, f"{target_name}.csv"), index=False)


<Figure size 640x480 with 0 Axes>

In [66]:
# the following targets are not in the EnQA-MSA results because they are too long to exceed the memory limit.
no_qa_results

['casp15_qa_results/H1157/qa.csv',
 'casp15_qa_results/H1137/qa.csv',
 'casp15_qa_results/H1168/qa.csv',
 'casp15_qa_results/H1111/qa.csv',
 'casp15_qa_results/H1168v1/qa.csv',
 'casp15_qa_results/H1114/qa.csv',
 'casp15_qa_results/H1114v2/qa.csv',
 'casp15_qa_results/H1185/qa.csv',
 'casp15_qa_results/H1171/qa.csv',
 'casp15_qa_results/T1115o/qa.csv',
 'casp15_qa_results/H1134/qa.csv']

In [67]:
def calc_correlation(df):
    # select the two columns we want to calculate correlation for
    selected_columns = df[["lDDToligo_True", "lDDToligo_EnQA-MSA"]]
    # calculate the correlation matrix
    corr_matrix = selected_columns.corr()
    # return the correlation value for the two columns
    return corr_matrix.loc["lDDToligo_True", "lDDToligo_EnQA-MSA"]  

def calc_loss(df):
    # sort the dataframe by predicted quality score in descending order
    first_ranked_model_by_lddt = df.sort_values("lDDToligo_EnQA-MSA", ascending=False).iloc[0]
    first_ranked_model_true_lddt = first_ranked_model_by_lddt["lDDToligo_True"]
    first_ranked_model_true_tmscore = first_ranked_model_by_lddt["TMscore"]
    first_ranked_model_true_qsglob = first_ranked_model_by_lddt["QSglob"]
    best_lddt = df.sort_values("lDDToligo_True", ascending=False).iloc[0]["lDDToligo_True"]
    best_tmscore = df.sort_values("TMscore", ascending=False).iloc[0]["TMscore"]
    best_qsglob = df.sort_values("QSglob", ascending=False).iloc[0]["QSglob"]
    # calculate the loss
    loss_lddt = best_lddt - first_ranked_model_true_lddt
    loss_tmscore = best_tmscore - first_ranked_model_true_tmscore
    loss_qsglob = best_qsglob - first_ranked_model_true_qsglob
    return  loss_lddt, loss_tmscore, loss_qsglob

In [68]:
stats = pd.DataFrame(columns=["target", "correlation", "loss-LDDT", "loss-TMscore", "loss-QSglob"])
for table_path in glob.glob(save_dir+"/*.csv"):
    table_name = table_path.split("/")[-1].split(".")[0]
    table = pd.read_csv(table_path)
    correlation = calc_correlation(table)
    loss_lddt, loss_tmscore, loss_qsglob = calc_loss(table)
    stats = stats.append({"target": table_name, "correlation": correlation, "loss-LDDT": loss_lddt , "loss-TMscore": loss_tmscore, "loss-QSglob": loss_qsglob}, ignore_index=True)
print(stats)
stats.to_csv("casp15_multimer_EnQA-MSA.csv", index=False,float_format='%.3f')
    

     target  correlation  loss-LDDT  loss-TMscore  loss-QSglob
0    T1132o     0.643353      0.004         0.000        0.000
1    T1124o     0.410851      0.002         0.018        0.010
2     H1141     0.416283      0.095         0.362        0.925
3     H1144     0.263351      0.009         0.001        0.021
4    T1109o     0.344015      0.062         0.072        0.463
5    T1127o     0.476944      0.039         0.010        0.026
6     H1140     0.205527      0.000         0.000        0.000
7     H1167     0.301865      0.056         0.176        0.166
8     H1106    -0.139552      0.496         0.429        0.870
9    T1110o     0.474538      0.014         0.009        0.005
10   T1113o     0.654506      0.072         0.114        0.086
11    H1151    -0.067309      0.058         0.027        0.029
12  H1166v1     0.259850      0.036         0.050        0.279
13  H1167v1     0.301737      0.056         0.228        0.122
14   T1121o     0.365142      0.019         0.305      

In [69]:
df = pd.read_csv("/root/EnQA/casp15_qa_comparison/T1132o.csv")

In [60]:
df.sort_values("TMscore", ascending=False)

,model,lDDToligo_True,TMscore,QSglob,lDDToligo_EnQA-MSA
0,T1132TS180_1o,0.928,0.997,0.954,0.929199
2,T1132TS462_3o,0.929,0.997,0.951,0.916992
1,T1132TS462_1o,0.932,0.997,0.953,0.923340
3,T1132TS180_2o,0.924,0.996,0.951,0.928711
5,T1132TS462_2o,0.928,0.996,0.949,0.920898
...,...,...,...,...,...
218,T1132TS234_2o,0.473,0.307,0.043,0.525879
220,T1132TS312_4o,0.684,0.285,0.040,0.701172
216,T1132TS493_3o,0.644,0.272,0.099,0.887695
219,T1132TS312_5o,0.685,0.235,0.042,0.704102
